<a href="https://colab.research.google.com/github/lakshitaa4/Diagnostic-Assistant/blob/main/Diagnostic_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***DIAGNOSTIC ASSISTANT***

In [ ]:
# @title Setup: Install Dependencies and Configure Gemini Pro API
# This section installs necessary libraries and sets up the connection to the Gemini Pro API.

# Install the google-generativeai library (for accessing Gemini Pro)
# !pip install -q -U google-generativeai
pip install -q -U google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 3.3 MB/s eta 0:00:00


In [62]:
# @title Import Necessary Libraries
from google import genai
from google.genai import types
import PIL.Image
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch
from google.colab import files
import cv2
import numpy as np
from PIL import Image
import pathlib
import mimetypes
client = genai.Client(api_key="your-API-KEY") # Please input your Google Ai Studio API Key

In [63]:
# @title Image Loading and Processing
def get_mime_type(file_path):
    # Determines the MIME type of a file based on its extension.
    mime_type, _ = mimetypes.guess_type(file_path)
    return mime_type if mime_type else "application/octet-stream"  # Default if unknown

def image_upload():
  uploaded_files = files.upload()  # Opens a file selection box

  images = []
  for file_name in uploaded_files.keys():
      # Open image using PIL
      pil_image = PIL.Image.open(file_name)
      mt = get_mime_type(file_name)

      # Convert image into correct format for Gemini API
      image_part = types.Part.from_bytes(
          data=pathlib.Path(file_name).read_bytes(),
          mime_type=mt # Adjust mime type if needed (e.g., "image/png")
      )

      # Append processed image to the list
      images.append(image_part)
      return images


In [70]:
# @title System Instruction and Output Format
# This section defines the system instruction for Gemini Pro.

# Define the system instruction for Gemini Pro
# This is like setting the "persona" or role of the AI"""

sys_ins = """
You are a highly skilled and experienced diagnostic assistant AI, designed to aid medical professionals in accurately
and efficiently diagnosing diseases. Your primary function is to analyze patient data and provide a ranked list of
potential diagnoses with supporting evidence, while also considering and ruling out alternative possibilities.
Analyze the following medical data and generate a list of potential diagnoses to assist medical professionals. Your response must be comprehensive and adhere to the following guidelines:

1.  **Data Analysis:** Carefully analyze all provided patient information, including medical history, symptoms, physical examination findings, and any relevant details.

2.  **Differential Diagnosis:** Generate a ranked list of the top 3-5 most likely diagnoses, along with a probability score (0-100%) for each. Ensure probabilities are realistic, reflecting the likelihood of each diagnosis given the data. Mention the primary diagnosis as "Primary Diagnoses" under this section.

3.  **Evidence-Based Reasoning (CRITICAL):** For each potential diagnosis, provide a clear and concise explanation of your reasoning. **Crucially, you MUST cite *specific* evidence directly from the provided patient data to support your conclusion.** Avoid general statements; instead, point to precise details in the patient's symptoms, history, or examination findings that support the diagnosis. Explain why you think the patient has this condition.

4.  **Alternative Diagnoses:** Discuss at least one alternative diagnosis that could also explain the patient's symptoms. Explain why that diagnosis is considered less likely than the top diagnoses, citing specific evidence against it.

5.  **Risk Factor Identification:** Identify any relevant risk factors present in the patient's history or lifestyle that may contribute to the potential diagnoses.

6.  **Severity Assessment:** Assess the potential severity of each diagnosis (e.g., mild, moderate, severe, life-threatening) and explain your reasoning.

7.  **Follow-Up Recommendations:** Suggest 2-3 relevant follow-up questions, examinations, or laboratory tests that could help to further refine the diagnosis and rule out other possibilities.

8.  **Bias Awareness and Mitigation (IMPORTANT):** Identify any potential biases that might be present in your reasoning due to limitations in the provided information, the prompt, or your training data. Present the biases as bullet points. Include recommendations that will give more information regarding each specific bias in order to potentially help to reduce the impact of it.
    Don't throw risk factors like 'anxiety' until 80 % sure
9. Make reference to the image output so that the information is correctly used. If the image output is not provided or does not aid you in providing more information to what can already be derived, then note that and add the following to 'relevant details': 'Analysis of images was not useful in this case.'

10. If the uploaded images are non-medical, respond with the message "This is not a valid medical image.  Please provide relevant patient details and medical images." Do not output JSON in that specific case.

11. If the prompt has a question, answer that in "ans_to_ques".
11.  **Structured Output (MANDATORY):** Provide your response in a JSON format that conforms to the following structure:

```json
{

  "patient_information": {
    "age": [Age],
    "symptoms": "[Symptoms]",
    "relevant_details": "[Any Other Pertinent Information]"
  },
  "IMAGE ANALYSIS": {
    "image type": "[Image Type]",
    "image_analysis": "[Image Analysis]"
  }
  "ans_to_ques": {"[Answer to Question]"}
  "differential_diagnosis": [
    {
      "diagnosis": "[Diagnosis Name]",
      "probability": [Probability Percentage (0-100)],
      "reasoning": "[Explanation with SPECIFIC EVIDENCE CITATIONS]",
      "severity": "[Severity Assessment]",
      "risk_factors": "[List of Risk Factors]"
    },
    {
      "diagnosis": "[Diagnosis Name]",
      "probability": [Probability Percentage (0-100)],
      "reasoning": "[Explanation with SPECIFIC EVIDENCE CITATIONS]",
      "severity": "[Severity Assessment]",
      "risk_factors": "[List of Risk Factors]"
    },
    ...
  ],
  "alternative_diagnoses": [
    {
      "diagnosis": "[Alternative Diagnosis Name]",
      "reasoning_against": "[Explanation of why this diagnosis is less likely]"
    }
  ],
  "follow_up_recommendations": [
    "[Follow-Up Recommendation 1]",
    "[Follow-Up Recommendation 2]",
    "[Follow-Up Recommendation 3]"
  ],
  "biases": [
   {"bias": "[Explanation of limitation]", "recommendation": "[Recommendation to obtain more information related to the specific bias]"}
  ],
  "confidence_level": [Confidence Percentage (0-100)],
  "important_note": "This information is intended for informational and educational purposes only and does not constitute medical advice. It is essential to consult with a qualified healthcare professional for any health concerns and should not be used as a substitute for a consultation with a healthcare provider."
}
Give output in JSON format
"""

In [ ]:
# @title Run Gemini Pro: Generate Content
# This section makes the call to the Gemini Pro API, passing in the patient data, symptoms, images (if any), and the system instruction.

images = image_upload()
prompt = input()
if not prompt:
  prompt = "Just give output based on image."
if len(images) == 0:
    response = client.models.generate_content(
  model="gemini-1.5-pro",
  contents=[prompt],
  config=types.GenerateContentConfig(
        system_instruction=sys_ins, temperature = 0.4))
else:
    response = client.models.generate_content(
    model="gemini-1.5-pro",
    contents=[prompt] + images,
    config=types.GenerateContentConfig(
          system_instruction=sys_ins, temperature = 0.4))

In [ ]:
# @title OUTPUT
print(response.text)